In [5]:
# später anpassen für colab und lokal

data_path = "./data"

In [8]:
!pip install pytorch-lightning


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.9/800.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 51.4 MB/s eta 0:00:00


In [9]:
from pytorch_lightning import LightningDataModule
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import os

class ConfigurableDataModule(LightningDataModule):
    """Class wraper für mit austauschbaren transforms"""
    def __init__(self, data_dir: str, batch_size: int, transform):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transform

    def setup(self, stage=None):
        # Erstellen des Datensatzes als Instanz von ImageFolder
        full_dataset = ImageFolder(root=self.data_dir, transform=self.transform)
        # Setzen der Trainingsset/Validierungsset Größe
        train_size = int(0.8 * len(full_dataset))
        val_size = len(full_dataset) - train_size
        # Zufälliges aufteilen in Training- und Validierungdatensatz
        self.train_dataset, self.val_dataset = random_split(full_dataset, [train_size, val_size])

    def train_dataloader(self):
        # Setzen des Traindataloader
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=os.cpu_count())

    def val_dataloader(self):
        # Setzen des Validation Dataloader
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=os.cpu_count())

In [10]:
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import pytorch_lightning as pl
import torch
import torchmetrics
from torchmetrics import MeanMetric
import torchvision
import torchvision.transforms.functional as F
import torchvision.utils as vutils
from torchvision.transforms.functional import to_pil_image, to_tensor
import random
import time
import shutil
from pathlib import Path

class BaseWasteClassifier(pl.LightningModule):
    CLASS_NAMES = ['Cardboard', 'Food Organics', 'Glass', 'Metal', 'Miscellaneous Trash', 'Paper', 'Plastic', 'Textile Trash', 'Vegetation']

    def __init__(self, num_classes: int, results_dir="results"):
        super().__init__()
        self.num_classes = num_classes
        # Get the class name of the model instance
        model_class_name = self.__class__.__name__
        # Initialize paths
        self.results_dir = Path(results_dir) / model_class_name
        self.models_dir = self.results_dir / "models"
        self.images_dir = self.results_dir / "images"
        self.plots_dir = self.results_dir / "plots"

        # Create directories if they don't exist
        self.results_dir.mkdir(parents=True, exist_ok=True)
        self.models_dir.mkdir(parents=True, exist_ok=True)
        self.images_dir.mkdir(parents=True, exist_ok=True)
        self.plots_dir.mkdir(parents=True, exist_ok=True)

        # Placeholder for the actual model
        self.model = None

        # Initialize metrics
        self.accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes, average='macro')
        self.precision = torchmetrics.Precision(task='multiclass', num_classes=num_classes, average='weighted')
        self.recall = torchmetrics.Recall(task='multiclass', num_classes=num_classes, average='weighted')
        self.f1_score = torchmetrics.F1Score(task='multiclass', num_classes=num_classes, average='weighted')

        # Initialize metrics for average loss
        self.train_loss_metric = torchmetrics.MeanMetric()
        self.val_loss_metric = torchmetrics.MeanMetric()
        # Initialize metrics for averaging
        self.avg_train_losses = []
        self.avg_val_losses = []


    def forward(self, x):
        raise NotImplementedError("This method should be overridden by subclasses.")

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        acc = self.accuracy(torch.argmax(logits, dim=1), y)
        precision = self.precision(torch.argmax(logits, dim=1), y)
        recall = self.recall(torch.argmax(logits, dim=1), y)
        f1 = self.f1_score(torch.argmax(logits, dim=1), y)  # Calculate F1 Score

        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        # Log precision, recall, and F1 Score
        self.log('train_precision', precision, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_recall', recall, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_f1', f1, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.train_loss_metric(loss)
        return loss

    def on_train_epoch_end(self):
        avg_train_loss = self.train_loss_metric.compute()
        self.log('epoch_avg_train_loss', avg_train_loss, on_epoch=True, prog_bar=True, logger=True)
        self.avg_train_losses.append(avg_train_loss.item())
        self.train_loss_metric.reset()

    def on_validation_start(self):
        self.clear_images_directory()

    def validation_step(self, batch, batch_idx):
        start_time = time.perf_counter()  # Start timing for inference speed

        x, y = batch
        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        acc = self.accuracy(torch.argmax(logits, dim=1), y)
        precision = self.precision(torch.argmax(logits, dim=1), y)
        recall = self.recall(torch.argmax(logits, dim=1), y)
        f1 = self.f1_score(torch.argmax(logits, dim=1), y)  # Calculate F1 Score

        inference_time = time.perf_counter() - start_time  # Stop timing for inference speed
        self.log('val_inference_time', inference_time, prog_bar=True, logger=True)

        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        # Log precision, recall, and F1 Score
        self.log('val_precision', precision, prog_bar=True)
        self.log('val_recall', recall, prog_bar=True)
        self.log('val_f1', f1, prog_bar=True)

        predictions = torch.argmax(logits, dim=1)  # Convert logits to predicted class indices

        if random.random() < 0.1:  # Log images randomly
            self.log_images_with_labels(x, y, predictions, batch_idx)

        self.val_loss_metric(loss)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def on_validation_epoch_end(self):
        # Compute the average losses for the current epoch
        avg_val_loss = self.val_loss_metric.compute()

        # Log the average losses
        self.log('epoch_avg_val_loss', avg_val_loss, on_epoch=True, prog_bar=True, logger=True)

        # Append the average losses to the lists for plotting
        self.avg_val_losses.append(avg_val_loss.item())

        # Reset the metrics for the next epoch
        self.val_loss_metric.reset()

        # Plot and save the loss curves
        self.plot_and_save_loss_curves()


    def log_images_with_labels(self, images, labels, predictions, batch_idx):
        """Save a batch of images with their actual and predicted labels, organized by class name and model class."""
        annotated_images = []
        for i, (image, label, prediction) in enumerate(zip(images, labels, predictions)):
            # Unnormalize the image for visualization
            image = self.unnormalize(image)  # Make sure to call with self if it's an instance method
            # Determine class name for the actual label
            actual_class_name = self.CLASS_NAMES[label.item()]

            # Ensure the class-specific directory exists within the model class directory
            image_dir = self.images_dir / actual_class_name
            image_dir.mkdir(parents=True, exist_ok=True)

            # Convert to PIL Image for easy manipulation
            pil_img = F.to_pil_image(image)

            # Annotate image with actual and predicted labels
            draw = ImageDraw.Draw(pil_img)
            annotation_text = f'Actual: {actual_class_name}, Predicted: {self.CLASS_NAMES[prediction.item()]}'
            draw.text((10, 10), annotation_text, fill="white")

            # Define the file path for saving the image within the specific class directory
            file_path = image_dir / f"epoch_{self.current_epoch}_batch_{batch_idx}_image_{i}.png"

            # Save the annotated image
            pil_img.save(file_path)

            # Convert back to tensor and add to list
            annotated_img = to_tensor(pil_img)
            annotated_images.append(annotated_img.unsqueeze(0))  # Add batch dimension

        # Stack all annotated images into a single tensor for logging
        annotated_images_tensor = torch.cat(annotated_images, dim=0)
        img_grid = torchvision.utils.make_grid(annotated_images_tensor, nrow=4)

        # Log the grid of annotated images
        self.logger.experiment.add_image(f'Validation Images, Batch {batch_idx}', img_grid, self.current_epoch)

    def clear_images_directory(self):
        if self.images_dir.exists() and self.images_dir.is_dir():
            for class_dir in self.images_dir.iterdir():
                if class_dir.is_dir():  # Ensure it's a directory
                    shutil.rmtree(class_dir)  # Delete the directory and all its contents

    def plot_and_save_loss_curves(self):
        plt.figure(figsize=(10, 6))
        plt.plot(range(len(self.avg_train_losses)), self.avg_train_losses, label='Average Training Loss')
        plt.plot(range(len(self.avg_val_losses)), self.avg_val_losses, label='Average Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Average Training and Validation Loss Over Epochs')
        plt.legend()
        plt.tight_layout()

        plot_path = self.plots_dir / "average_loss_curves.png"
        plt.savefig(plot_path)
        plt.close()

    def unnormalize(self, image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
        """Revert normalization of an image tensor."""
        image = image.clone()  # Clone the tensor to avoid in-place operations
        for t, m, s in zip(image, mean, std):
            t.mul_(s).add_(m)  # Multiply by std and add mean
        return image

    def configure_optimizers(self):
        # Subclasses can override this if needed
        optimizer = torch.optim.SGD(self.model.parameters(), lr=0.001, momentum=0.9)
        return optimizer


In [11]:
import os
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger


def train_model(model, data_module, log_dir="tb_logs", max_epochs=50, logger_name="model_logs", callbacks=[]):
    # Configure the ModelCheckpoint callback
    checkpoint_callback = ModelCheckpoint(
        dirpath=model.models_dir,
        filename='{epoch}-{val_loss:.2f}',
        save_top_k=2,  # Save only the best checkpoint
        verbose=True,
        monitor='val_loss',  # Monitor validation loss (change to val_acc or any other metric as needed)
        mode='min',  # 'min' for loss (use 'max' for accuracy)
    )
    callbacks.append(checkpoint_callback)
    # Starten das Trainingsprozesses
    logger = TensorBoardLogger(log_dir, name=logger_name)
    trainer = Trainer(max_epochs=max_epochs, logger=logger, callbacks=callbacks)
    trainer.fit(model, datamodule=data_module)

## SimpleCNN

In [ ]:
import torch.nn as nn

class SimpleCNN(BaseWasteClassifier):
    def __init__(self, num_classes=9, lr=1e-3):
        self.lr = 1e-3
        super().__init__(num_classes)
        self.model = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 512),  # Bestätigt, dass dies für eine Eingabegröße von 224x224 korrekt ist
            nn.ReLU(),
            nn.Dropout(0.5), # Overfitting vermeiden
            nn.Linear(512, num_classes) # lineare Schicht auf die Klassen
        )


    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

### Train SimpleCNN

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=tb_logs/

In [ ]:
from torchvision import transforms
# Definiere die Transformationspipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Skaliere alle Bilder auf 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping

data_module = ConfigurableDataModule(data_dir=data_path, batch_size=32, transform=transform)

callbacks = [
    # Stoppt des Training wenn sich val Metrik nicht verbessert
    EarlyStopping(
        monitor="val_acc",  # Die Metrik beobachtet wird
        mode="max",  # Maximiert die Genauigkeit
        patience=1,  # "Wartet" 1 Epoche ohne Verbesserung
    )
]
model = SimpleCNN(num_classes=9, lr=1e-3)
train_model(model, data_module, log_dir="tb_logs", max_epochs=50, logger_name="simple_CNN", callbacks=callbacks)

## InceptionV3

In [ ]:
from torchvision.models import inception_v3

class InceptionWasteClassifier(BaseWasteClassifier):
    def __init__(self, num_classes=9):
        super().__init__(num_classes)
        self.model = inception_v3(weights="Inception_V3_Weights.DEFAULT")
        self.model.fc = torch.nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        if self.training:
            outputs = self.model(x)
            return outputs.logits, outputs.aux_logits
        else:
            return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)  # This could be either a tensor or a tuple of tensors
        logits = outputs if isinstance(outputs, torch.Tensor) else outputs[0]  # Select logits
        loss = torch.nn.functional.cross_entropy(logits, y)
        acc = self.accuracy(torch.argmax(logits, dim=1), y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

In [ ]:
from torchvision.transforms.functional import InterpolationMode
# Define the transformations as per Inception V3's requirements
transform = transforms.Compose([
    transforms.Resize(342, interpolation=InterpolationMode.BILINEAR),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping

data_module = ConfigurableDataModule(data_dir=data_path, batch_size=32, transform=transform)

callbacks = [
    # Stoppt des Training wenn sich val Metrik nicht verbessert
    EarlyStopping(
        monitor="val_acc",  # Die Metrik beobachtet wird
        mode="max",  # Maximiert die Genauigkeit
        patience=1,  # "Wartet" 1 Epoche ohne Verbesserung
    )
]
model = InceptionWasteClassifier(num_classes=9)
train_model(model, data_module, log_dir="tb_logs", max_epochs=50, logger_name="inceptionv3", callbacks=callbacks)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\Felix\anaconda3\envs\cudatest\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:652: Checkpoint directory C:\Users\Felix\Desktop\RealWaste\results\InceptionWasteClassifier\models exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type                | Params
----------------------------------------------------------
0 | accuracy          | MulticlassAccuracy  | 0     
1 | precision         | MulticlassPrecision | 0     
2 | recall            | MulticlassRecall    | 0     
3 | f1_score          | MulticlassF1Score   | 0     
4 | train_loss_metric | MeanMetric          | 0     
5 | val_loss_metric   | MeanMetric          | 0     
6 | model             | Inception3          | 25.1 M
----------------------------------------------------------
25.1 M    Trainable params
0      

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\Felix\anaconda3\envs\cudatest\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


c:\Users\Felix\anaconda3\envs\cudatest\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 0: 100%|██████████| 119/119 [01:25<00:00,  1.40it/s, v_num=1, train_loss_step=1.030, train_acc_step=0.507, val_inference_time=0.100, val_loss=0.849, val_acc=0.744, val_precision=0.803, val_recall=0.776, val_f1=0.768, epoch_avg_val_loss=0.849, train_loss_epoch=1.570, train_acc_epoch=0.441]

c:\Users\Felix\anaconda3\envs\cudatest\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric MeanMetric was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
Epoch 0, global step 119: 'val_loss' reached 0.84873 (best 0.84873), saving model to 'C:\\Users\\Felix\\Desktop\\RealWaste\\results\\InceptionWasteClassifier\\models\\epoch=0-val_loss=0.85.ckpt' as top 2


Epoch 1:   0%|          | 0/119 [00:00<?, ?it/s, v_num=1, train_loss_step=1.030, train_acc_step=0.507, val_inference_time=0.100, val_loss=0.849, val_acc=0.744, val_precision=0.803, val_recall=0.776, val_f1=0.768, epoch_avg_val_loss=0.849, train_loss_epoch=1.570, train_acc_epoch=0.441]          

c:\Users\Felix\anaconda3\envs\cudatest\Lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001E6D05779C0>
Traceback (most recent call last):
  File "c:\Users\Felix\anaconda3\envs\cudatest\Lib\site-packages\torch\utils\data\dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "c:\Users\Felix\anaconda3\envs\cudatest\Lib\site-packages\torch\utils\data\dataloader.py", line 1437, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
                                   ^^^^^^^^^^^^^^^^^^^^
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


## DenseNet201

In [ ]:
from torchvision.models import densenet201

class DenseNet201Classifier(BaseWasteClassifier):
    def __init__(self, num_classes=9):
        super().__init__(num_classes)
        self.model = densenet201(weights="DenseNet201_Weights.DEFAULT")
        self.model.classifier = torch.nn.Linear(self.model.classifier.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

In [ ]:
from torchvision.transforms.functional import InterpolationMode
# Define the transformations as per Inception V3's requirements
transform = transforms.Compose([
    transforms.Resize(256, interpolation=InterpolationMode.BILINEAR),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping

data_module = ConfigurableDataModule(data_dir=data_path, batch_size=32, transform=transform)

callbacks = [
    # Stoppt des Training wenn sich val Metrik nicht verbessert
    EarlyStopping(
        monitor="val_acc",  # Die Metrik beobachtet wird
        mode="max",  # Maximiert die Genauigkeit
        patience=1,  # "Wartet" 1 Epoche ohne Verbesserung
    )
]
model = DenseNet201Classifier(num_classes=9)
train_model(model, data_module, log_dir="tb_logs", max_epochs=50, logger_name="densenet201", callbacks=callbacks)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\Felix\anaconda3\envs\cudatest\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:652: Checkpoint directory C:\Users\Felix\Desktop\RealWaste\results\DenseNet201Classifier\models exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type                | Params
----------------------------------------------------------
0 | accuracy          | MulticlassAccuracy  | 0     
1 | precision         | MulticlassPrecision | 0     
2 | recall            | MulticlassRecall    | 0     
3 | f1_score          | MulticlassF1Score   | 0     
4 | train_loss_metric | MeanMetric          | 0     
5 | val_loss_metric   | MeanMetric          | 0     
6 | model             | DenseNet            | 18.1 M
----------------------------------------------------------
18.1 M    Trainable params
0         

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\Felix\anaconda3\envs\cudatest\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


c:\Users\Felix\anaconda3\envs\cudatest\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 0:   3%|▎         | 3/119 [00:31<20:27,  0.09it/s, v_num=6, train_loss_step=2.260, train_acc_step=0.180, train_precision_step=0.215, train_recall_step=0.125, train_f1_step=0.122]  

## Resnet50

In [ ]:
from torchvision.models import resnet50

class ResNet50Classifier(BaseWasteClassifier):
    def __init__(self, num_classes=9):
        super().__init__(num_classes)
        self.model = resnet50(weights="ResNet50_Weights.DEFAULT")
        self.model.fc = torch.nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)


In [ ]:
from torchvision.transforms.functional import InterpolationMode

transform = transforms.Compose([
    transforms.Resize(256, interpolation=InterpolationMode.BILINEAR),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping

data_module = ConfigurableDataModule(data_dir=data_path, batch_size=32, transform=transform)

callbacks = [
    # Stoppt des Training wenn sich val Metrik nicht verbessert
    EarlyStopping(
        monitor="val_acc",  # Die Metrik beobachtet wird
        mode="max",  # Maximiert die Genauigkeit
        patience=1,  # "Wartet" 1 Epoche ohne Verbesserung
    )
]
model = ResNet50Classifier(num_classes=9)
train_model(model, data_module, log_dir="tb_logs", max_epochs=50, logger_name="resnet50", callbacks=callbacks)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\Felix/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:03<00:00, 33.4MB/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\Felix\anaconda3\envs\cudatest\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:652: Checkpoint directory C:\Users\Felix\Desktop\RealWaste\results\ResNet50Classifier\models exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type                | Params
----------------------------------------------------------
0 | accuracy          | MulticlassAccuracy  | 0     
1 | precision         | MulticlassPrecision | 0     
2 | recall            | MulticlassRecall    | 0     
3 | f1_score          | MulticlassF1Score   | 0     
4 | train_loss_metric | MeanMetric          | 0     
5 | val_lo

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\Felix\anaconda3\envs\cudatest\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


c:\Users\Felix\anaconda3\envs\cudatest\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 0: 100%|██████████| 119/119 [01:14<00:00,  1.59it/s, v_num=2, train_loss_step=1.320, train_acc_step=0.497, train_precision_step=0.723, train_recall_step=0.600, train_f1_step=0.640, val_inference_time=0.0874, val_loss=1.190, val_acc=0.672, val_precision=0.729, val_recall=0.697, val_f1=0.672, epoch_avg_val_loss=1.190, train_loss_epoch=1.730, train_acc_epoch=0.390, train_precision_epoch=0.451, train_recall_epoch=0.445, train_f1_epoch=0.396]

...

Epoch 7: 100%|██████████| 119/119 [01:49<00:00,  1.09it/s, v_num=2, train_loss_step=0.0475, train_acc_step=1.000, train_precision_step=1.000, train_recall_step=1.000, train_f1_step=1.000, val_inference_time=0.0691, val_loss=0.311, val_acc=0.898, val_precision=0.918, val_recall=0.898, val_f1=0.898, epoch_avg_val_loss=0.313, train_loss_epoch=0.0952, train_acc_epoch=0.981, train_precision_epoch=0.988, train_recall_epoch=0.983, train_f1_epoch=0.983, epoch_avg_train_loss=0.134]

Epoch 7, global step 952: 'val_loss' reached 0.31137 (best 0.31137), saving model to 'C:\\Users\\Felix\\Desktop\\RealWaste\\results\\ResNet50Classifier\\models\\epoch=7-val_loss=0.31.ckpt' as top 2


Epoch 8: 100%|██████████| 119/119 [01:49<00:00,  1.09it/s, v_num=2, train_loss_step=0.0202, train_acc_step=1.000, train_precision_step=1.000, train_recall_step=1.000, train_f1_step=1.000, val_inference_time=0.0707, val_loss=0.300, val_acc=0.896, val_precision=0.922, val_recall=0.898, val_f1=0.899, epoch_avg_val_loss=0.302, train_loss_epoch=0.0718, train_acc_epoch=0.989, train_precision_epoch=0.993, train_recall_epoch=0.989, train_f1_epoch=0.990, epoch_avg_train_loss=0.0951]

Epoch 8, global step 1071: 'val_loss' reached 0.29980 (best 0.29980), saving model to 'C:\\Users\\Felix\\Desktop\\RealWaste\\results\\ResNet50Classifier\\models\\epoch=8-val_loss=0.30.ckpt' as top 2


Epoch 8: 100%|██████████| 119/119 [01:49<00:00,  1.08it/s, v_num=2, train_loss_step=0.0202, train_acc_step=1.000, train_precision_step=1.000, train_recall_step=1.000, train_f1_step=1.000, val_inference_time=0.0707, val_loss=0.300, val_acc=0.896, val_precision=0.922, val_recall=0.898, val_f1=0.899, epoch_avg_val_loss=0.302, train_loss_epoch=0.0718, train_acc_epoch=0.989, train_precision_epoch=0.993, train_recall_epoch=0.989, train_f1_epoch=0.990, epoch_avg_train_loss=0.0951]


# VGG16

In [49]:
from torchvision.models import vgg16

class vgg16Classifier(BaseWasteClassifier):
    def __init__(self, num_classes=9):
        super().__init__(num_classes)
        self.model = vgg16(weights="VGG16_Weights.DEFAULT")
        self.model.classifier[6] = torch.nn.Linear(self.model.classifier[6].in_features, num_classes)

    def forward(self, x):
        return self.model(x)

In [50]:
from torchvision.transforms.functional import InterpolationMode

transform = transforms.Compose([
    transforms.Resize(256, interpolation=InterpolationMode.BILINEAR),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [51]:
from pytorch_lightning.callbacks import EarlyStopping

data_module = ConfigurableDataModule(data_dir=data_path, batch_size=32, transform=transform)

callbacks = [
    # Stoppt des Training wenn sich val Metrik nicht verbessert
    EarlyStopping(
        monitor="val_acc",  # Die Metrik beobachtet wird
        mode="max",  # Maximiert die Genauigkeit
        patience=1,  # "Wartet" 1 Epoche ohne Verbesserung
    )
]
model = vgg16Classifier(num_classes=9)
train_model(model, data_module, log_dir="tb_logs", max_epochs=50, logger_name="vgg16", callbacks=callbacks)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


FileNotFoundError: [Errno 2] No such file or directory: './data'